In [475]:
import numpy as np
import itertools as it
from thpoker.core import * # Combo, Hand, Table

from collections import Counter

In [476]:
from platform import python_version

print(python_version())

3.8.5


# Create a Deck of Cards

In [477]:
class Deck:
    
    def __init__(self):
        self.cards = []
        self.deck_dict = {}
        for value in ["2","3","4","5","6","7","8","9","T","J","Q","K","A"]:
            for suit in ["h","d","c","s"]:
                self.cards.append(value + suit)
        for card in self.cards:
            self.deck_dict[Card(card)] = card
        for bottom_straight in ["1s","1d","1c","1h"]:
            self.deck_dict[Card(bottom_straight)] = "A" + bottom_straight[1]

In [478]:
test = Deck()
test.deck_dict

{2♥: '2h',
 2♦: '2d',
 2♣: '2c',
 2♠: '2s',
 3♥: '3h',
 3♦: '3d',
 3♣: '3c',
 3♠: '3s',
 4♥: '4h',
 4♦: '4d',
 4♣: '4c',
 4♠: '4s',
 5♥: '5h',
 5♦: '5d',
 5♣: '5c',
 5♠: '5s',
 6♥: '6h',
 6♦: '6d',
 6♣: '6c',
 6♠: '6s',
 7♥: '7h',
 7♦: '7d',
 7♣: '7c',
 7♠: '7s',
 8♥: '8h',
 8♦: '8d',
 8♣: '8c',
 8♠: '8s',
 9♥: '9h',
 9♦: '9d',
 9♣: '9c',
 9♠: '9s',
 T♥: 'Th',
 T♦: 'Td',
 T♣: 'Tc',
 T♠: 'Ts',
 J♥: 'Jh',
 J♦: 'Jd',
 J♣: 'Jc',
 J♠: 'Js',
 Q♥: 'Qh',
 Q♦: 'Qd',
 Q♣: 'Qc',
 Q♠: 'Qs',
 K♥: 'Kh',
 K♦: 'Kd',
 K♣: 'Kc',
 K♠: 'Ks',
 A♥: 'Ah',
 A♦: 'Ad',
 A♣: 'Ac',
 A♠: 'As',
 1♠: 'As',
 1♦: 'Ad',
 1♣: 'Ac',
 1♥: 'Ah'}

# Creating a Class for an NLH Table

In [479]:
class NLHTable:
    
    def __init__(self, n_seats=9):
        self.n_seats = n_seats
        
    def deal_hand(self):
        
        # deal to each player
        card_deck = Deck().cards
        dealt_cards = np.random.choice(card_deck, 2*self.n_seats, replace=False)
        hands = [dealt_cards[i:(i+2)] for i in range(0, 2*self.n_seats, 2)]
        
        
        # deal the runout
        cards_left = list(it.chain(*[ [k]*v for k,v in (Counter(card_deck) - Counter(dealt_cards)).items() ]))
        board = np.random.choice(cards_left, 5, replace=False)
        
        return hands, board
    
    def quads_or_better(self, hand, board):
        hand_eval = "/".join(hand)
        board_eval = "/".join(board)
        five_card_hand = Combo(hand=Hand(hand_eval), table=Table(board_eval))
        five_card_hand_as_str = [Deck().deck_dict[card_as_str] for card_as_str in five_card_hand.cards]
        # this ensures that the correct hand is achieved AND that both whole cards are in play
        if five_card_hand.name in ['four of a kind', 'straight flush'] and all([Deck().deck_dict[Card(whole_card)] in five_card_hand_as_str for whole_card in hand]):
            return True
        return False
    
    def bad_beat_hit(self, hands, board):
        for hand in hands:
            # check if a bad beat has been hit for a hand dealt
            pass

In [480]:
hands, board = NLHTable(9).deal_hand()
hands, board

([array(['6d', 'Qs'], dtype='<U2'),
  array(['2d', 'Qc'], dtype='<U2'),
  array(['5d', 'Tc'], dtype='<U2'),
  array(['Ah', '7c'], dtype='<U2'),
  array(['7h', 'Qd'], dtype='<U2'),
  array(['8d', 'Kd'], dtype='<U2'),
  array(['4d', '4s'], dtype='<U2'),
  array(['6h', 'Jd'], dtype='<U2'),
  array(['Ac', 'Td'], dtype='<U2')],
 array(['Th', '9d', 'Kc', '8h', '3s'], dtype='<U2'))

# Test High Hand Qualifications

In [481]:
# this is just 4 of a kind aces
hand1 = ["Ah", "As"]
board1 = ["Ac","Ad","Kd","2d","3s"]

# this is NOT a proper bad beat hand because you do not use 
# both whole cards to make your five card hand (AAAAK)
hand2 = ["Ah", "2s"]
board2 = ["Ac","Ad","Kd","2d","3s"]

# While this is not a pocket pair, the Ace DOES play in 
# the five card hand (3333A) so this qualifies for bad beat
hand3 = ["3h", "As"]
board3 = ["3c","3d","Kd","2d","3s"]

# this a straight flush requiring both whole cards and should work
hand4 = ["3h", "4h"]
board4 = ["5h","9d","6h","2d","7h"]

# this a straight flush NOT requiring both whole cards and should work
hand5 = ["3h", "As"]
board5 = ["5h","4h","6h","2d","7h"]

# this is the hand a fish plays
hand6 = ["3h", "4s"]
board6 = ["Tc","9d","Kd","2d","8s"]

hand7 = ['4s', '3h']
board7 = ['6s', '5s', '3s', 'Tc', '7s']

assert NLHTable().quads_or_better(hand1, board1)
assert not NLHTable().quads_or_better(hand2, board2)
assert NLHTable().quads_or_better(hand3, board3)
assert NLHTable().quads_or_better(hand4, board4)
assert not NLHTable().quads_or_better(hand5, board5)
assert not NLHTable().quads_or_better(hand6, board6)
assert not NLHTable().quads_or_better(hand7, board7)

# Creating the Simulation

In [484]:
def bad_beat_simulation(n_tables=1000, n_hands=100000):
    bad_beat_list = []
    hand_counter = 0
    bad_beat_counter = 0
    while hand_counter <= n_hands:
        for table in range(n_tables):
            current_table = NLHTable()
            hands, board = current_table.deal_hand()
            current_table_high_hand_counter = 0
            for hand in hands:
                if current_table.quads_or_better(hand, board):
                    current_table_high_hand_counter += 1
                    print(hand, board)
                if current_table_high_hand_counter >= 2:
                    bad_beat_counter += 1
                    bad_beat_list.append((hands, board))
                    # this break is important because it is possible that 3 hands qualify for bad beat
                    # but we only want to count this as 1 bad beat because the jackpot doesn't get
                    # paid out twice in this EXTREMELY rare situation
                    break
        hand_counter += n_tables
        print(hand_counter)
    return bad_beat_counter, bad_beat_counter/n_hands, bad_beat_list

In [485]:
bad_beat_simulation()

['Qh' 'Qd'] ['6d' '7h' 'Qs' 'Qc' '3c']
['Ks' 'Js'] ['As' 'Ts' 'Qs' 'Qh' 'Ah']
['7h' '7d'] ['7c' '4s' 'Qc' '6d' '7s']
['Ac' 'As'] ['Ad' '8d' 'Kh' 'Qc' 'Ah']
['9c' '9s'] ['Qs' '9d' '5c' '9h' 'Js']
['Tc' '7c'] ['Jd' '9c' '8c' '6c' 'Qd']
['6c' '6d'] ['6s' 'Qc' '6h' 'Js' 'Kd']
['Ts' 'Th'] ['Tc' 'Td' '8c' '2s' '9s']
['3d' '3h'] ['3c' '3s' '5d' '6s' '7s']
1000
['Jh' 'Kh'] ['2c' '7s' 'Js' 'Jd' 'Jc']
2000
['Jc' 'Kd'] ['Jd' '4c' '4s' 'Jh' 'Js']
['As' 'Th'] ['Td' 'Ac' 'Tc' 'Ts' '3h']
['5d' '5c'] ['2c' 'Jc' '5h' '6s' '5s']
['2c' '2d'] ['2s' '3h' 'Js' '9h' '2h']
['2h' '2s'] ['8d' 'Jc' 'Jd' '2d' '2c']
['Jd' 'Kd'] ['Jh' '6h' '7s' 'Jc' 'Js']
['3c' '3s'] ['3d' '3h' '9c' 'Td' '9s']
['9c' '8c'] ['9h' '9d' '4d' '5d' '9s']
['2c' '3c'] ['7d' '5c' '9h' 'Ac' '4c']
['4h' '4d'] ['4s' '2s' '4c' '8h' '8c']
['Jh' 'Qs'] ['Qc' '7h' 'Qh' 'Qd' 'Th']
3000
['6c' 'Jh'] ['3s' '5c' '6d' '6s' '6h']
['Ah' 'Ac'] ['As' '5d' 'Ad' '4c' '7h']
['As' 'Ac'] ['Ah' 'Ad' 'Kh' 'Kd' '3s']
['Jh' 'Jc'] ['7d' 'Js' 'Th' '8s' 'Jd']
['Qs' 'Jc'

['Qc' 'Tc'] ['Qs' 'Qh' '6d' '8c' 'Qd']
['3d' '3s'] ['Js' '3c' '3h' 'Kc' 'Td']
['Qc' '9c'] ['3h' '2d' '8c' 'Tc' 'Jc']
['As' '9h'] ['2h' '9s' '9c' '9d' 'Ac']
25000
['Td' 'Ts'] ['Qh' '3c' '7c' 'Tc' 'Th']
['7c' '8c'] ['4c' '6c' 'Kd' '8d' '5c']
['7s' '7c'] ['7d' '3s' 'Qs' '7h' '9c']
['6d' 'Jh'] ['6c' '2h' '6s' '6h' '7d']
26000
['9h' 'Ks'] ['9s' 'Jc' '9d' '9c' '8d']
['Jc' 'Jd'] ['7d' '8d' 'Js' 'Qc' 'Jh']
['Jc' 'Js'] ['Jh' 'Jd' '5s' '5d' '8c']
['8d' '9s'] ['7d' '8h' '8s' '2h' '8c']
['5c' 'Ks'] ['Kh' 'Kc' '2c' 'Kd' '3s']
['Ac' 'Ah'] ['Ad' 'Jd' '3c' 'As' '8c']
['Qs' 'Qc'] ['5s' '7h' 'Qh' 'Qd' 'Tc']
['6d' 'Ac'] ['Td' '9s' '6s' '6c' '6h']
['6d' '7h'] ['5s' '7c' '7d' '3d' '7s']
['7s' '9s'] ['9d' '9c' '3h' '7h' '9h']
27000
['As' 'Td'] ['Ts' '4s' 'Th' 'Tc' '4d']
['Qs' 'Qc'] ['Qd' 'Qh' '9h' '4c' 'Jh']
['Qc' 'Qd'] ['8s' '6h' 'Kc' 'Qs' 'Qh']
['7h' 'Jc'] ['5h' 'Jd' '5c' 'Js' 'Jh']
['Jd' 'Js'] ['Jh' '3s' 'Jc' 'Kc' '6c']
['2c' 'Kd'] ['2s' '8c' '2h' '6c' '2d']
28000
['Ac' 'Ah'] ['2d' '3d' 'As' '8s' 'Ad']
[

49000
['6h' '8d'] ['8s' '5c' '6c' '8h' '8c']
['Jh' 'Jd'] ['5h' '5d' 'Qh' 'Jc' 'Js']
['Jh' 'Th'] ['6s' '7h' '9h' '4s' '8h']
['Th' 'Ts'] ['Qd' 'Tc' 'Td' 'Ad' 'Ac']
50000
['9s' 'Kc'] ['9h' '3d' '2s' '9c' '9d']
['Ah' '9s'] ['9h' '9c' '3d' '3s' '9d']
['9h' '9d'] ['Ah' '9c' '9s' '7h' '5c']
['7d' '8d'] ['6d' '5s' 'Ad' '5d' '4d']
['Th' 'Kh'] ['Jh' 'Ah' '7d' 'Kd' 'Qh']
['7s' '7d'] ['Qc' '7h' '9s' 'Jc' '7c']
['Qs' '8s'] ['2c' 'Qd' 'Qc' '4h' 'Qh']
['7c' 'Ks'] ['Kd' 'Kc' 'Kh' '3c' '2s']
['3d' '3c'] ['Qd' 'Kc' '5h' '3h' '3s']
['5s' 'As'] ['4s' '8c' '3s' '2s' '2c']
['9c' '9h'] ['8d' '9s' 'Td' '9d' '6d']
['6c' '9c'] ['Tc' 'Ad' '2h' '8c' '7c']
['4d' '4h'] ['4c' 'Kd' '4s' '6c' '7h']
51000
['Ac' 'Ad'] ['Ks' 'As' 'Qc' 'Qd' 'Ah']
['Qc' 'Kc'] ['Kd' 'Ks' 'Kh' 'Js' '4d']
['3d' '6d'] ['2d' '5c' '4d' '5d' 'Ts']
['2c' '8c'] ['8d' '2h' '2s' '7s' '2d']
['5d' '5c'] ['Ac' 'Ad' '5h' 'Th' '5s']
['Tc' 'Kc'] ['Jc' '3h' 'Ac' 'Qc' '6d']
['Qh' '5c'] ['9d' '7d' '5h' '5d' '5s']
['Kh' 'Ac'] ['Kd' 'Ks' 'Kc' '7h' 'Qs']
['Ad' '

['Qd' 'Js'] ['5h' 'Qc' '7h' 'Qh' 'Qs']
['5h' '5d'] ['5c' 'Qd' '5s' '8s' '2h']
['5d' 'Jh'] ['9h' '5s' '5h' '3c' '5c']
['Ts' 'Tc'] ['Td' 'Jh' 'Th' '7h' 'Ad']
['Ts' '8s'] ['5c' '9s' 'Qs' '5h' 'Js']
['Ad' 'Kc'] ['Kd' 'Ks' 'Qs' '2h' 'Kh']
['6c' '6h'] ['6s' 'As' '6d' '8d' '5s']
['Qd' 'Qh'] ['9h' '7c' '7h' 'Qs' 'Qc']
['9d' 'Ad'] ['Ac' '2s' 'As' 'Ah' '9c']
72000
['7s' '7h'] ['7c' '2d' '2s' '7d' 'Ks']
['Th' 'Tc'] ['Ts' 'Qs' 'Kd' 'Td' '6s']
['7c' 'As'] ['7d' '7s' '3c' 'Jd' '7h']
['4d' '4h'] ['Qc' '4c' '6d' '4s' '7h']
['3c' 'Ad'] ['5s' '2d' '3d' '3h' '3s']
['9s' 'Kc'] ['4c' 'Ks' 'Kh' '8d' 'Kd']
['5h' '5c'] ['5d' '5s' 'Ac' 'Jc' 'Ah']
73000
['6d' 'Td'] ['9c' '9d' '7d' '2h' '8d']
['8c' '9c'] ['Kh' '7c' 'Tc' '4c' '6c']
['9h' 'Qh'] ['Kh' '6d' 'Th' 'Jh' 'Td']
['Th' 'Ad'] ['9h' 'Td' 'Tc' '4h' 'Ts']
['Qh' '8s'] ['Qc' 'Qd' 'Qs' '6d' '3h']
['Qs' 'Kd'] ['7h' 'Qd' 'Kc' 'Qc' 'Qh']
74000
['9c' 'Ah'] ['9d' '9s' '8c' '9h' '3c']
['Kh' 'Kd'] ['Kc' '7c' '8d' 'Ks' 'Tc']
['3h' '5h'] ['9c' 'Ks' '6h' '2h' '4h']
['6h' '

['Qh' 'Th'] ['4s' '9h' 'Jc' 'Jh' '8h']
['3h' '3c'] ['8s' '5d' '3d' 'Tc' '3s']
['8h' '9h'] ['Jh' '7h' '5s' 'Th' '6s']
['Kd' 'Kc'] ['8d' '6h' 'Ac' 'Kh' 'Ks']
['Ts' '8h'] ['8s' '8d' '5s' '8c' '6s']
['7s' '6s'] ['Ts' '8s' '4s' 'Tc' '5s']
97000
['Qs' 'Qc'] ['5s' '3h' '7s' 'Qd' 'Qh']
['Ts' 'Tc'] ['Th' 'Qd' 'Td' 'Ac' '8s']
['Kd' 'Kc'] ['3h' '3d' 'Kh' '4h' 'Ks']
['8s' '7s'] ['6s' '3h' '9s' 'Jd' '5s']
['5c' 'Tc'] ['4s' '7c' '5d' '5h' '5s']
['7h' '4h'] ['6h' '5h' 'Ts' '3c' '3h']
['3d' '8s'] ['5c' '3c' '6d' '3s' '3h']
['2h' '4h'] ['4c' '3h' '6h' 'Ad' '5h']
['Jc' 'Kc'] ['Qc' 'Tc' '9c' '4d' '3s']
['4s' 'Ac'] ['Kc' '4d' 'Tc' '4h' '4c']
['Qs' '4h'] ['Qc' '4d' '4s' '4c' '2d']
98000
['9s' '9c'] ['Kh' '9d' '9h' '7s' 'Ah']
['7c' '7h'] ['Jd' 'Ks' 'Qd' '7s' '7d']
['Qs' 'Qd'] ['9c' 'Qc' 'Qh' '2d' '6h']
['9d' 'Jd'] ['Kd' 'Js' 'Td' 'Qd' '2c']
['3d' '3c'] ['Jh' '3h' '3s' '5h' '7d']
['9h' '9d'] ['7s' '3d' '9c' '9s' '3s']
['6s' '6c'] ['6d' '5s' '4c' '6h' '3h']
['6h' 'Qd'] ['6s' 'Ts' '6c' '6d' 'Qc']
99000
['4c' '

(0, 0.0, [])